In [6]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import os

In [7]:
# Stable Diffusion 모델 불러오기
model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("mps")  # M1, M2, M3 칩에서 MPS를 사용해 성능 최적화

Loading pipeline components...: 100%|██████████| 6/6 [00:10<00:00,  1.70s/it]


In [8]:
# LoRA 적용을 위해 모델 로드 및 미세 조정
from diffusers import UNet2DConditionModel
from torch import nn

class LoRAUNet(nn.Module):
    def __init__(self, unet: UNet2DConditionModel, r: int = 4):
        super().__init__()
        self.unet = unet
        # UNet의 모든 가중치에 대해 LoRA 적용
        for name, module in self.unet.named_modules():
            if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
                weight = module.weight
                # Low-rank approximation을 위한 행렬
                self.unet.add_module(f"{name}_lora", nn.Parameter(torch.randn(r, weight.shape[0])))
    
    def forward(self, *args, **kwargs):
        return self.unet(*args, **kwargs)

In [9]:
# 기존의 UNet 모델을 LoRA 적용된 모델로 변경
pipe.unet = LoRAUNet(pipe.unet)

TypeError: torch.FloatTensor is not a Module subclass

## 방법을 다시 알아야함